# Caso Práctico - ENGIE

*Jose Enrique Zafra Mena*

## Introducción


## Carga de datos

In [139]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl

In [140]:
facturas_df=pd.read_excel('facturas.xlsx')
mapping_df=pd.read_excel('Mapping.xlsx')

# renombramos los Deals
CTV_df=pd.read_excel('JVLNG_CTV_STOK_MARZO_2024_05_23_14_12.xlsx')
IAC_df=pd.read_excel('MEDG2_IAC_GETRA_MARZO_2024_05_23_14_12.xlsx')
AOC2_df=pd.read_excel('SUGS2_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')
AOC_df=pd.read_excel('SUGST_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')


In [141]:
mapping_df.head()

,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
1,TVB,PLANTAS,JVLNG,ICC,GETRA
2,TVB,PLANTAS,JVLNG,ICC,GETRA
3,TVB,PLANTAS,JVLNG,ICC,GETRA
4,TVB,PLANTAS,JVLNG,ICC,GETRA


## Transformación de los datos


### Mapping
Quitamos los duplicados y todas las Commoditys que no necesitamos: nos quedamos solo con CTV, IAC, AOC. También escribimos bien los tipos de Servicios Facturados.

In [142]:
mapping_df=mapping_df.query("Commodity in ['CTV', 'IAC', 'AOC'] and Portfolio in ['MEDG2','JVLNG', 'SUGST', 'SUGS2']").drop_duplicates()

# cambiamos los nombres
fixes = {
    'ExtracciÃ³n':'Extracción',
    'InyecciÃ³n':'Inyección'
}

mapping_df['Servicio facturado'] = mapping_df['Servicio facturado'].replace(fixes)

mapping_df.head(15)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
16,TVB,Almacenamiento TVB,JVLNG,CTV,STOK
25,AVB,Servicio agregado AVB,SUGST,AOC,STOK
26,AVB,Extracción,SUGST,AOC,STOK
27,AVB,Salida PVB a AVB,SUGST,AOC,STOK
28,AVB,Inyección,SUGST,AOC,STOK
29,AVB,Entrada PVB desde AVB,SUGST,AOC,STOK
30,AVB,Almacenamiento AVB,SUGST,AOC,STOK
31,AVB,Servicio agregado AVB,SUGS2,AOC,STOK
32,AVB,Extracción,SUGS2,AOC,STOK


### Facturas
Quitamos columnas que no son relevantes, ajustamos formato de las fechas, y reordenamos columnas:

In [143]:
mapping_df.rename(columns={'Servicio facturado': 'Servicio Facturado'}, inplace=True)

facturas_new_df = facturas_df.loc[:, ['NumeroFactura', 'Importe', 'FechaFactura', 'Origen', 'ServicioFacturado']]

facturas_new_df.rename(columns={'NumeroFactura': 'Número de Factura', 'Importe':'Importe Factura', 'FechaFactura':'Fecha Factura',
                                'ServicioFacturado':'Servicio Facturado'}, inplace=True)

facturas_new_df['Fecha Factura'] = pd.to_datetime(facturas_new_df['Fecha Factura'], format="%d/%m/%y")

#facturas_new_df['Fecha Factura 2'] = facturas_new_df['Fecha Factura'].dt.strftime('%d/%m/%Y') # con esto dejan de ser datetime64, pero es mejor para verlo directamente

facturas_new_df.head(20)

,Número de Factura,Importe Factura,Fecha Factura,Origen,Servicio Facturado
0,2024264672,123195.13,2024-03-04,AVB,Servicio agregado AVB
1,2024264671,111963.62,2024-03-04,AVB,Servicio agregado AVB
2,2324004231,503763.47,2024-03-06,C.I. Almería,Entrada PVB
3,2024164592,1375.12,2024-03-04,TVB,Almacenamiento TVB
4,2024164591,1787.43,2024-03-04,TVB,Almacenamiento TVB
5,2024164590,1930.23,2024-03-04,TVB,Almacenamiento TVB
6,2024164589,1472.13,2024-03-04,TVB,Almacenamiento TVB
7,2024164588,1965.66,2024-03-04,TVB,Almacenamiento TVB
8,2024164587,1041.67,2024-03-04,TVB,Almacenamiento TVB
9,2024164586,1030.97,2024-03-04,TVB,Almacenamiento TVB


### Deals
Los Deals tienen columnas que no necesitamos para la conciliación. Necesitamos solo algo que lo identifique (ID), su importe y una categoría

In [144]:
# para renombrar las columnas, usamos las columnas de CTV
CTV_min_df = CTV_df.loc[:,['Id', 'TotalQuantity', 'TradeDate', 'CommodityType']]

# renombramos
CTV_min_df.rename(columns={'TotalQuantity':'Quantity'} , inplace=True)

# ahora escogemos las columnas que queremos de cada Deal
IAC_min_df = IAC_df.loc[:,['Id', 'Quantity', 'TradeDate', 'CommodityType']]
AOC_min_df = AOC_df.loc[:,['Id', 'Quantity', 'TradeDate', 'CommodityType']]
AOC2_min_df = AOC2_df.loc[:,['Id', 'Quantity', 'TradeDate', 'CommodityType']]

# TODO
# añadir columnas de strike y tal

# añadir columna de coste (calculado)


Vamos a conectar el Mapping con cada Deal, ya que queremos comparar con las facturas.

Para conectar Mapping con CTV usamos Commodity==CTV

Para conectar Mapping con IAC usamos Commodity==IAC

Para conectar Mapping con AOC usamos Commodity==AOC & Portofolio: SUGST

Para conectar Mapping con AOC2 usamos Commodity==AOC & Portofolio: SUGS2


In [145]:
# Añadimos las columnas de Portfolio y Commodity correspondientes 
# en los Deals para poder relacionarlos con el Mapping

CTV_min_df['Portfolio']='JVLNG'
CTV_min_df['Commodity']='CTV'

IAC_min_df['Portfolio']='MEDG2'
IAC_min_df['Commodity']='IAC'

AOC_min_df['Portfolio']='SUGST'
AOC_min_df['Commodity']='AOC'

AOC2_min_df['Portfolio']='SUGS2'
AOC2_min_df['Commodity']='AOC'

AOC_min_df.head()

,Id,Quantity,TradeDate,CommodityType,Portfolio,Commodity
0,41888250,800000.000,2024-02-22T00:00:00+01:00,EG,SUGST,AOC
1,38290460,924.588,2023-04-20T00:00:00+02:00,EG,SUGST,AOC
2,38249489,352.340,2023-04-17T00:00:00+02:00,EG,SUGST,AOC
3,38054569,3910.694,2023-03-24T00:00:00+01:00,EG,SUGST,AOC
4,36525979,800000.000,2022-11-16T00:00:00+01:00,EG,SUGST,AOC


#### Unión de los *Deals*

In [146]:
# ahora unimos todos los deals en un solo dataframe 
deals_df=pd.concat([CTV_min_df, IAC_min_df, AOC_min_df, AOC2_min_df], ignore_index=True)

# cambiamos el formato de fecha
deals_df['TradeDate'] = (pd.to_datetime(deals_df['TradeDate'], utc=True ,dayfirst=False, format='%Y-%m-%dT%H:%M:%S%z'))

# cambiamos el nombre de la columna
deals_df.rename(columns={'TradeDate':'Fecha Deal'} , inplace=True)

deals_df.head(100)

,Id,Quantity,Fecha Deal,CommodityType,Portfolio,Commodity
0,42563586,90000.0,2024-04-28 22:00:00+00:00,EG,JVLNG,CTV
1,42518127,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV
2,42518126,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV
3,42518125,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV
4,42518124,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV
...,...,...,...,...,...,...
58,37671348,900.0,2023-02-21 23:00:00+00:00,EG,SUGS2,AOC
59,37356869,400000.0,2022-11-20 23:00:00+00:00,EG,SUGS2,AOC
60,37186479,100000.0,2023-01-11 23:00:00+00:00,EG,SUGS2,AOC
61,36876402,100000.0,2022-12-18 23:00:00+00:00,EG,SUGS2,AOC


#### Merge con mapping

In [147]:
# ahora unimos cada uno con mapping
deals_mapped_df = (pd.merge(deals_df, mapping_df,on=['Portfolio', 'Commodity'], how='inner').drop_duplicates())


test=deals_mapped_df.shape[0]-deals_df.shape[0]
print('deals añadidos:',test)

#deals_mapped_df.head(1000)
deals_mapped_df.head(160)

## SALEN DE MÁS PORQUE LA RELACIÓN NO ES ÚNICA, PARA UN MISMO CANAL (PORTOFOLIO+COMMODITY), 
## HAY DISTINTOS SERVICIOS FACTURADOS           

deals añadidos: 95


,Id,Quantity,Fecha Deal,CommodityType,Portfolio,Commodity,Origen,Servicio Facturado,DealType
0,42563586,90000.0,2024-04-28 22:00:00+00:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
1,42518127,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
2,42518126,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
3,42518125,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
4,42518124,50000.0,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
...,...,...,...,...,...,...,...,...,...
153,36572988,100000.0,2022-11-20 23:00:00+00:00,EG,SUGS2,AOC,AVB,Extracción,STOK
154,36572988,100000.0,2022-11-20 23:00:00+00:00,EG,SUGS2,AOC,AVB,Salida PVB a AVB,STOK
155,36572988,100000.0,2022-11-20 23:00:00+00:00,EG,SUGS2,AOC,AVB,Inyección,STOK
156,36572988,100000.0,2022-11-20 23:00:00+00:00,EG,SUGS2,AOC,AVB,Entrada PVB desde AVB,STOK


#### Problemas del merge
Para un mismo ID de un Deal, vemos que puede provenir de diferentes servicios facturados. Hacemos un nuevo dataframe con una columna para cada tipo de servicio facturado. Esto solo tiene sentido porque sabemos que no hay muchos tipos de servicios facturados.

In [148]:
deals_copia_df = deals_mapped_df[['Id', 'Quantity', 'Fecha Deal', 'CommodityType', 'Portfolio','Commodity', 'Origen','DealType','Servicio Facturado']]

# añadimos las columnas con get_dummies (para que sean binarias)
servicios_dummies=pd.get_dummies(deals_copia_df['Servicio Facturado'])

servicios_dummies.head()

,Almacenamiento AVB,Almacenamiento TVB,Entrada PVB,Entrada PVB desde AVB,Extracción,Inyección,Salida PVB a AVB,Servicio agregado AVB
0,False,True,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False


In [149]:
# los añadimos y quitamos la columna de Servicios,
deals_dummies_df = pd.concat([deals_copia_df, servicios_dummies], axis=1)
deals_dummies_df.drop(columns=['Servicio Facturado'], inplace=True)

deals_dummies_df.head(100)

,Id,Quantity,Fecha Deal,CommodityType,Portfolio,Commodity,Origen,DealType,Almacenamiento AVB,Almacenamiento TVB,Entrada PVB,Entrada PVB desde AVB,Extracción,Inyección,Salida PVB a AVB,Servicio agregado AVB
0,42563586,90000.0000,2024-04-28 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
1,42518127,50000.0000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
2,42518126,50000.0000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
3,42518125,50000.0000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
4,42518124,50000.0000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,40089592,281.7058,2023-10-08 22:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,False,False,False,False,False,True,False,False
96,40089592,281.7058,2023-10-08 22:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,False,False,False,True,False,False,False,False
97,40089592,281.7058,2023-10-08 22:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,True,False,False,False,False,False,False,False
98,38410189,1178.1630,2023-05-02 22:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,False,False,False,False,False,False,False,True


In [150]:
# ahora tenemos que tomar, para cada ID único, ver de todos los servicios de los que viene
# se puede hacer con un for, pero groupby es más eficiente

entradas = ['Almacenamiento AVB', 'Almacenamiento TVB', 'Entrada PVB', 'Entrada PVB desde AVB','Extracción', 'Inyección', 'Salida PVB a AVB', 'Servicio agregado AVB']

# agrupamos por id y usamos any,
entradas_grouped = deals_dummies_df.groupby('Id')[entradas].any()
no_entradas = [ 'Quantity', 'Fecha Deal', 'CommodityType', 'Portfolio','Commodity', 'Origen', 'DealType']
no_entradas_grouped = deals_dummies_df.groupby('Id')[no_entradas].first()


deals_final_df = no_entradas_grouped.join(entradas_grouped).reset_index()

deals_final_df.head(100)

,Id,Quantity,Fecha Deal,CommodityType,Portfolio,Commodity,Origen,DealType,Almacenamiento AVB,Almacenamiento TVB,Entrada PVB,Entrada PVB desde AVB,Extracción,Inyección,Salida PVB a AVB,Servicio agregado AVB
0,31915597,NaN,2021-08-11 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
1,34044001,429860.700,2022-03-08 23:00:00+00:00,EG,SUGST,AOC,AVB,STOK,True,False,False,True,True,True,True,True
2,34833331,NaN,2022-05-23 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
3,34835329,NaN,2022-05-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
4,34935869,4372.765,2022-05-31 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,42518124,50000.000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
59,42518125,50000.000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
60,42518126,50000.000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
61,42518127,50000.000,2024-04-24 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False


Con esto tenemos un dataframe de Deals con Ids únicos, y columnas que nos indican desde qué servicios provienen.

#### Filtrado de fechas
Los trades no ocurren a la vez que las facturas. En general, las facturas se generan después. Como solo tenemos las fecturas de principio de marzo, esa es la cota superior. Probamos con cotas inferiores hasta tener el mismo número de deals que de facturas (14):

In [151]:
deals_marzo_df = deals_final_df[(deals_final_df['Fecha Deal'] >= '2023-7-04') & (deals_final_df['Fecha Deal'] <= '2024-03-31')] 
print(deals_marzo_df.loc[deals_marzo_df['Almacenamiento TVB']==True,'Quantity'].sum() )

deals_marzo_df.head(15)

262000.0


,Id,Quantity,Fecha Deal,CommodityType,Portfolio,Commodity,Origen,DealType,Almacenamiento AVB,Almacenamiento TVB,Entrada PVB,Entrada PVB desde AVB,Extracción,Inyección,Salida PVB a AVB,Servicio agregado AVB
30,39057923,NaN,2023-07-04 22:00:00+00:00,RES,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
31,39705811,NaN,2023-09-05 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
32,39942100,NaN,2023-09-26 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
33,39980344,NaN,2023-09-28 22:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
34,40089592,281.70580,2023-10-08 22:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,True,False,False,True,True,True,True,True
35,40094436,897.70579,2023-10-16 22:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,True,False,False,True,True,True,True,True
36,41888250,800000.00000,2024-02-21 23:00:00+00:00,EG,SUGST,AOC,AVB,STOK,True,False,False,True,True,True,True,True
37,41985669,50000.00000,2024-02-28 23:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False
38,41996650,15000.00000,2024-02-29 23:00:00+00:00,EG,SUGS2,AOC,AVB,STOK,True,False,False,True,True,True,True,True
39,42238676,91000.00000,2024-03-25 23:00:00+00:00,EG,JVLNG,CTV,TVB,STOK,False,True,False,False,False,False,False,False


In [152]:
## borrar
facturas_new_df.head(15)

,Número de Factura,Importe Factura,Fecha Factura,Origen,Servicio Facturado
0,2024264672,123195.13,2024-03-04,AVB,Servicio agregado AVB
1,2024264671,111963.62,2024-03-04,AVB,Servicio agregado AVB
2,2324004231,503763.47,2024-03-06,C.I. Almería,Entrada PVB
3,2024164592,1375.12,2024-03-04,TVB,Almacenamiento TVB
4,2024164591,1787.43,2024-03-04,TVB,Almacenamiento TVB
5,2024164590,1930.23,2024-03-04,TVB,Almacenamiento TVB
6,2024164589,1472.13,2024-03-04,TVB,Almacenamiento TVB
7,2024164588,1965.66,2024-03-04,TVB,Almacenamiento TVB
8,2024164587,1041.67,2024-03-04,TVB,Almacenamiento TVB
9,2024164586,1030.97,2024-03-04,TVB,Almacenamiento TVB


## Análisis de los datos

### Facturas

In [153]:
# Número total de facturas
numero_total_facturas = facturas_new_df['Número de Factura'].count()
print('Número total de facturas: ',numero_total_facturas)
# Importe total de facturas 
importe_total_facturas = facturas_new_df['Importe Factura'].sum()
print('Importe total de facturas: '  + f'{importe_total_facturas:.2f}'+' €' )

# Distribución por categoría

### NO APLICABLE CREO (luego al unir con los deals??)

Número total de facturas:  14
Importe total de facturas: 811357.98 €


### Deals

In [154]:
# Vamos a ver si en Deals (sin mappear) hay IDs repetidos 
## esto implicaria que los canales son redundantes

num_ID_deals = deals_df['Id'].count()-deals_df['Id'].nunique()
print('IDs totales - ID unicos: ', num_ID_deals)

# los canales no son redundantes

# Vemos tras el mappeado,
num_ID_deals_mapped = deals_mapped_df['Id'].count()-deals_mapped_df['Id'].nunique()
print('IDs totales - ID unicos: ', num_ID_deals_mapped)


IDs totales - ID unicos:  0
IDs totales - ID unicos:  95


In [155]:
# Número total de Deals
numero_total_deals = deals_df['Quantity'].count()
numero_total_deals_mapped = deals_mapped_df['Quantity'].count()
numero_total_deals_mapped_unicos = deals_mapped_df['Quantity'].nunique()

### unique counts?

print('Número total de deals: ',numero_total_deals)
print('Número total de deals_mapped: ',numero_total_deals_mapped)
print('Número total de deals_mapped (únicos): ',numero_total_deals_mapped_unicos)

# Volumen total de Deals

# Distribución por Categorías

Número total de deals:  50
Número total de deals_mapped:  145
Número total de deals_mapped (únicos):  30


## Conciliación

### Merge de Facturas con los *Deals*

### Unificación de fechas y filtrado

In [156]:
# por ultimo cambio el nombre y el orden de las columnas para verlo mejor
facturas_conciliadas_Marzo_df.rename(columns={'NumeroFactura': 'Número de Factura', 'FechaFactura':'Fecha Factura', 'Id':'ID Deal', 'LegId':'LegID', 
                                              'Quantity':'Importe Deal', 'Importe':'Importe Factura', 'TradeDate':'Fecha Deal'}, inplace=True)

orden = ['Número de Factura', 'ID Deal', 'Importe Factura', 'Importe Deal', 'Fecha Factura', 'Fecha Deal', 'LegID',
         'Origen', 'Servicio Facturado', 'LegType', 'Category', 'Portfolio', 'Commodity', 'DealType', 'Conciliado', 'Observaciones']

facturas_conciliadas_Marzo_df = facturas_conciliadas_Marzo_df.reindex(columns=orden)

facturas_conciliadas_Marzo_df.head(100)

NameError: name 'facturas_conciliadas_Marzo_df' is not defined

## Propuesta de Valor